In [2]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
import pandas as pd

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
conf_dw = {
      'url':'jdbc:mysql://localhost:3306/eltmysql?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigMysql',
      'password':'bigMysql1234@'   
      }
}

conf_dm = {
      'url':'jdbc:mysql://localhost:3306/etlmysqlDM?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigDM',
      'password':'bigDM1234@'   
      }
}

In [5]:
# table 저장 / 반환 관련한 함수 구성
# config : db 연결 정보
# spark.read.jdbc() : db에서 명시된 테이블을 반환받는 모듈
def find_data(config, table_name) :
    return spark.read.jdbc(url= config['url'], table=table_name, properties=config['props'])

def save_data(config, df, table_name) :
    return df.write.jdbc(url= config['url'], table=table_name, mode='append' , properties=config['props'])

In [6]:
# 데이터 불러오기
restaurant = find_data(conf_dw, 'RESTAURANT_LIST')

In [7]:
restaurant.columns

['addr',
 'food',
 're_rank1',
 're_rank2',
 're_rank3',
 're_rank4',
 're_rank5',
 're_visitor',
 'store_name',
 'tel',
 'std_day']

In [8]:
restaurant.count()

2081

In [9]:
# id컬럼 추가
restaurant = restaurant.withColumn('id', monotonically_increasing_id())

In [10]:
restaurant.show()

+--------------------------------+--------+--------------------+------------------------+-------------------------+------------------------+------------------------+----------------------------------+-----------------------+--------------+----------+---+
|                            addr|    food|            re_rank1|                re_rank2|                 re_rank3|                re_rank4|                re_rank5|                        re_visitor|             store_name|           tel|   std_day| id|
+--------------------------------+--------+--------------------+------------------------+-------------------------+------------------------+------------------------+----------------------------------+-----------------------+--------------+----------+---+
|        서울 중구 창경궁로 62-29|비빔냉면|음식이 맛있어요 3782|      매장이 넓어요 1358|특별한 메뉴가 있어요 1213|         양이 많아요 858|            친절해요 773|       왜⸌◦̈⃝⸍ʷʰʸ 미슐랭 맛집인...|                 우래옥|  02-2265-0151|2024-01-19|  0|
|        서울 강남구 학동로 305-3|비빔냉면|음식

## restaurant - dropduplicates

In [11]:
# food와 store_name 컬럼을 통틀어서 중복된 행 찾기
duplicate_rows = restaurant.groupBy("food", "store_name", 'addr').count().filter(col("count") > 1)

# 결과 출력
duplicate_rows.show(truncate=False)

+----------+---------------------------------------+----------------------------------------------+-----+
|food      |store_name                             |addr                                          |count|
+----------+---------------------------------------+----------------------------------------------+-----+
|감자전    |김창훈포차                             |서울 광진구 능동로13길 65 1층 김창훈포차      |2    |
|감자탕    |인생감자탕 합정점                      |서울 마포구 양화로1길 18 1층                  |2    |
|짜장면    |한양중식                               |서울 송파구 송파대로 167 A동 205호            |2    |
|해물찜    |약수동해물텀벙                         |서울 중구 다산로10길 11                       |2    |
|족발      |예가족발 본점                          |서울 강동구 양재대로 1424 1층                 |2    |
|감자탕    |청년감자탕 송파 본점                   |서울 송파구 오금로19길 5 1층                  |2    |
|곱창구이  |박진주할머니김치곱창전골 서울대입구본점|서울 관악구 남부순환로234길 19 1층            |2    |
|순대      |두루올순대                             |서울 은평구 은평로9길 13 캐스텔가든 두루올순대|2    |
|삼계탕    |백세삼계탕 

In [12]:
# 중복 제거
restaurant_drop_duplicate = restaurant.dropDuplicates(["food", "store_name", 'addr'])
restaurant_drop_duplicate.show(truncate=False)

+----------------------------------------------------------------------+--------+--------------------+-------------------+------------------------+------------------------+-------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------+--------------+----------+---+
|addr                                                                  |food    |re_rank1           

In [13]:
# 중복제거된지 확인
# food와 store_name 컬럼을 통틀어서 중복된 행 찾기
duplicate_rows = restaurant_drop_duplicate.groupBy("food", "store_name", 'addr').count().filter(col("count") > 1)

# 결과 출력
duplicate_rows.show(truncate=False)

+----+----------+----+-----+
|food|store_name|addr|count|
+----+----------+----+-----+
+----+----------+----+-----+



In [14]:
# 하나만 남기고 중복제거되었는지 재확인
# 조건에 맞는 행 필터링
filtered_rows = restaurant_drop_duplicate.filter((col("food") == "감자전") & (col("store_name") == "김창훈포차"))

# 결과 출력
print("Filtered Rows:")
filtered_rows.show(truncate=False)

Filtered Rows:


+----------------------------------------+------+--------------------+----------------------+------------+-----------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------------+----------+---+
|addr                                    |food  |re_rank1            |re_rank2              |re_rank3    |re_rank4         |re_rank5             |re_visitor                                                                                                                                                                        |store_name|tel           |std_day   |id |
+----------------------------------------+------+--------------------+----------------------+------------+-----------------+---------------------+------------------------------------------------------------------------------------------------------------------------

In [15]:
restaurant_drop_duplicate.count()
restaurant_drop_duplicate.columns

2065

['addr',
 'food',
 're_rank1',
 're_rank2',
 're_rank3',
 're_rank4',
 're_rank5',
 're_visitor',
 'store_name',
 'tel',
 'std_day',
 'id']